In [45]:
#!/usr/bin/env python
import snowflake.connector
import pandas as pd
import numpy as np
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
###
# https://docs.snowflake.com/en/user-guide/python-connector-example.html
from glob import glob

1) Reading RAW data and data Cleaning and Transformation Process 

In [46]:
filenames = glob('raw_datasets/*/20*.csv',recursive=True) 
print(len(filenames))
count=0
for file in filenames:
    # Read the data files individually 
    data = pd.read_csv(file)
    print('Reading file ==>' ,file)
    #Cleaning the data base for null values and replacing the with corrresponing valuse and droping unwanted columns
    data=ET_load(data)
    print('Cleaned file ==>',file)
    #Saving the cleaned file to to new directory
    p = 'processed_datasets/'+file[21:] 
    data.to_csv(p,header=None,index=False)
    print('Saving File ==>',file)
    count=count+1
    print(count,'/',len(filenames),'Processed!')


534
Reading file ==> raw_datasets/2019-03/2019-03-hampshire-street.csv
~~Empty values replaced!!
~~Dataset Cleaned
~~Duplicated Entries Removed
Cleaned file ==> raw_datasets/2019-03/2019-03-hampshire-street.csv
Saving File ==> raw_datasets/2019-03/2019-03-hampshire-street.csv
##########################################
Reading file ==> raw_datasets/2019-03/2019-03-south-yorkshire-street.csv
~~Empty values replaced!!
~~Dataset Cleaned
~~Duplicated Entries Removed
Cleaned file ==> raw_datasets/2019-03/2019-03-south-yorkshire-street.csv
Saving File ==> raw_datasets/2019-03/2019-03-south-yorkshire-street.csv
##########################################
Reading file ==> raw_datasets/2019-03/2019-03-essex-street.csv
~~Empty values replaced!!
~~Dataset Cleaned
~~Duplicated Entries Removed
Cleaned file ==> raw_datasets/2019-03/2019-03-essex-street.csv
Saving File ==> raw_datasets/2019-03/2019-03-essex-street.csv
##########################################
Reading file ==> raw_datasets/2019-03/2019

**Counting number of output files 

534


2) Connecting to Snowflake 

In [48]:
# Create main connection context
con = snowflake.connector.connect(
    user='anuragvuppala',
    password='Snow@Flake44',
    account='vx24548.europe-west2.gcp'
)

Connection Validification

In [55]:
c = con.cursor()
c.execute("SELECT current_version()").fetchone()
c.execute("USE ROLE ACCOUNTADMIN") 

Database and table creation

In [61]:
c.execute("CREATE WAREHOUSE IF NOT EXISTS STREET_CRIME_WH")
c.execute("CREATE DATABASE IF NOT EXISTS STREET_CRIME_DB")
c.execute("USE WAREHOUSE STREET_CRIME_WH")
c.execute('USE DATABASE STREET_CRIME_DB')
c.execute('USE SCHEMA PUBLIC')

In [63]:
c.execute('CREATE TABLE IF NOT EXISTS "CRIME_DATA_TABLE" (crime_id STRING, month STRING, reported_by STRING, fall_within STRING, longitude FLOAT, latitude FLOAT, location STRING, lsoa_code STRING, lsoa_name STRING, crime_type STRING, last_outcome STRING, id integer not null constraint unique_id primary key not enforced)')

In [66]:

c.execute("PUT file://processed_datasets/20*.csv @%crime_data_table")



In [67]:
c.execute("COPY INTO CRIME_DATA_TABLE")
c.execute("Remove @%crime_data_table")
print('Romoved files form Stageing area!')


Counting Total number of crime by crime type 

In [69]:
crime_count_type=c.execute("SELECT crime_type,COUNT(crime_type) FROM CRIME_DATA_TABLE GROUP BY crime_type ORDER BY COUNT(crime_type) DESC; ").fetchall()
crime_count_type

[('Violence and sexual offences', 1862757),
 ('Anti-social behaviour', 890303),
 ('Criminal damage and arson', 548354),
 ('Other theft', 513148),
 ('Vehicle crime', 446240),
 ('Public order', 430076),
 ('Burglary', 383960),
 ('Shoplifting', 357758),
 ('Drugs', 174768),
 ('Other crime', 103665),
 ('Theft from the person', 102781),
 ('Bicycle theft', 85304),
 ('Robbery', 85237),
 ('Possession of weapons', 46757)]

Counting Total number of crime count for every street

In [70]:
crime_count_street = c.execute("SELECT location,COUNT(crime_type) FROM CRIME_DATA_TABLE GROUP BY location ORDER BY COUNT(crime_type) DESC; ").fetchall()
len(crime_count_street)

240333

In [71]:
con.close()

~~Empty values replaced!!
~~UniqueID Created!!
~~Dataset Cleaned!!
~~Duplicated Entries Removed


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,UN_ID,u_id
0,NA,2021-01,British Transport Police,British Transport Police,-0.271604,50.834400,On or near Shoreham-By-Sea (Station),E01031365,Adur 002D,Criminal damage and arson,NA,5072350021735546583,0 NA\n2 NA\n3 NA\n4 NA...
2,NA,2021-01,British Transport Police,British Transport Police,-0.271604,50.834400,On or near Shoreham-By-Sea (Station),E01031365,Adur 002D,Public order,NA,5072350021735546583,0 NA\n2 NA\n3 NA\n4 NA...
3,NA,2021-01,British Transport Police,British Transport Police,-0.640432,50.830800,On or near Barnham (Station),E01031393,Arun 003A,Public order,NA,5072350021735546583,0 NA\n2 NA\n3 NA\n4 NA...
4,NA,2021-01,British Transport Police,British Transport Police,-0.546132,50.810200,On or near Littlehampton (Station),E01031428,Arun 011B,Drugs,NA,5072350021735546583,0 NA\n2 NA\n3 NA\n4 NA...
5,NA,2021-01,British Transport Police,British Transport Police,-0.546132,50.810200,On or near Littlehampton (Station),E01031428,Arun 011B,Public order,NA,5072350021735546583,0 NA\n2 NA\n3 NA\n4 NA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141,NA,2021-01,British Transport Police,British Transport Police,-4.314210,55.879900,On or near Hyndland (Station),NA,NA,Violence and sexual offences,NA,5072350021735546583,0 NA\n2 NA\n3 NA\n4 NA...
2142,NA,2021-01,British Transport Police,British Transport Police,-3.188535,55.952496,On or near Edinburgh Waverley (Station),NA,NA,Other crime,NA,5072350021735546583,0 NA\n2 NA\n3 NA\n4 NA...
2143,NA,2021-01,British Transport Police,British Transport Police,-4.250850,55.862400,On or near Glasgow Queen Street (Station),NA,NA,Other crime,NA,5072350021735546583,0 NA\n2 NA\n3 NA\n4 NA...
2145,NA,2021-01,British Transport Police,British Transport Police,-4.310590,55.862242,On or near Govan Subway Station (Station),NA,NA,Other crime,NA,5072350021735546583,0 NA\n2 NA\n3 NA\n4 NA...
